# Overview : Paillier cryptosystem

In this notebook, we explore a simple example to explore homomorphic encryption technique using [Paillier](https://en.wikipedia.org/wiki/Paillier_cryptosystem) cryptosystem. The Pailier cryptosystem is an efficient additive partial homomorphic encryption system that is based on the composite residuosity class problem. This means that given only the ciphertexts on $m_{1}$ and $m_{2}$ along with the same public key, anyone can calculate the cipher text on $m_{1} + m_{2}$. This method works very well in privacy preservation for financial scenarios where the transactions are mainly related to addition and substraction operations on the amount or balance. 

## Public key encryption scheme 

The basic public key encryption scheme has three stages:
1. generate a public-private key pair.
2. encrypt a number.
3. decrypt a number.

The **helper functions** that we use are:

1. $gcd(x,y)$ outputs the greatest common divisor of $x$ and $y$.
2. $lcm(x,y)$ outputs the least common multiple of $x$ and $y$.

**Key Generation**

The process of key generation are as follows,

1. Pick two large prime numbers $p$ and $q$, randomly and independently. Confirm that $gcd(pq,(p-1)(q-1))$ is $1$. If not, start again.
2. Compute $n = pq$.
3. Define function, $L(x) = \frac{(x-1)}{n}$.
4. Compute $\lambda$ as $lcm(p-1, q-1)$.
5. Pick a random integer $g$ in the set $Z^*_{n^2}$ (integers between $1$ and $n^2$).
6. Calculate the [modular multiplicative inverse](https://en.wikipedia.org/wiki/Modular_multiplicative_inverse) $\mu = (L(g^\lambda\ mod \ n^2))^{-1} mod \ n$. If $\mu$ does not exist, start again from step 1. (For understanding, if $x \ mod \ y = r$ then, $x \ = \ dividend$, $y \ = \ divisor$ and $r \ = \ remainder$)
7. The public key is $(n,g)$. We use this from encryption.
8. The private key is $\lambda$. We use this for decryption.

**Encryption**

The steps for encryption are as follows,

1. Let $m$ be a message to be encrypted where $0\le m<n$.
2. Pick a random number $r$ in the range $0<r<n$.
3. Compute $cipher \ text, c = g^m . r^n mod (n^2)$

**Decryption** 

1. Let $c$ be the cipher text to decrypt, where $c \in Z^*_{n^2}$.
2. We compute the plain text message as: $m = L(c^\lambda mod \ n^2).\mu \ mod(n)$

## Public Key Encryption Scheme (Example)

**Key Generation**

1. Pick $p \ = \ 13$ and $q \ = \ 17$. (These are previously tested numbers that satisfy the condition.)
2. Compute $n \ = \ 221$.
3. Compute $\lambda \ = \ 48$.
4. Pick $g \ = \ 4886$.
5. Compute $\mu \ = \ 159$.


**Encryption**
1. Set $m_{1} \ = \ 123$.
2. Pick $r_{1} \ = \ 666$.
3. Compute $cipher \ text, c_{1} \ = \ 25889 \ mod \ 221^2$.

**Decryption***
1. $m_{decrypted} = 123 \ mod \ 221 \ = 123$. (Same as $m_{1})$

## Using [python-paillier](https://pypi.org/project/phe/)

In this section, we will see how we can implement a partial homomorphic encryption technique using paillier python pachage.

In [1]:
from phe import paillier
import statistics

**Assigning two random numbers**

In [2]:
num1 = 30
num2 = 40

**Generating public and private keys and encrypting the numbers**

In [3]:
pub_key, priv_key = paillier.generate_paillier_keypair()
cipher_num1, cipher_num2 = pub_key.encrypt(num1), pub_key.encrypt(num2)

**Adding two encrypted numbers together**

In [4]:
result = cipher_num1 + cipher_num2
result = priv_key.decrypt(result)

**Decrypting the result**

In [5]:
print("Add two encrypted numbers together:", result)

Add two encrypted numbers together: 70


**Adding a scalar to the encrypted number and decrypting the result**

In [6]:
result = cipher_num1 + 5
result = priv_key.decrypt(result)

In [7]:
print("Add an encrypted number to a number:", result)

Add an encrypted number to a number: 35


Calculate the time taken to perform addition.

In [8]:
print("Homomorphic Addition takes:")
he_add =  %timeit -n2 -o (cipher_num1 + cipher_num2)
median_he = statistics.median(he_add.all_runs)
print("Median for HE addition {}".format(median_he))
print("Vector Addition takes:")
v_add = %timeit -n2 -o (num1 + num2)
res = he_add.best / v_add.best
median_vec = statistics.median(v_add.all_runs)
print("Median for Vector Addition {}".format(median_vec))
result_median = median_he/ median_vec

Homomorphic Addition takes:
92.8 µs ± 3.4 µs per loop (mean ± std. dev. of 7 runs, 2 loops each)
Median for HE addition 0.0001842719502747059
Vector Addition takes:
102 ns ± 62.3 ns per loop (mean ± std. dev. of 7 runs, 2 loops each)
Median for Vector Addition 1.448206603527069e-07


In [9]:
print("Vector Addition is {} times faster than Homomorphic Addition".format(res))
print("Median Vector Addition is {} times better than Median Homomorphic Addition".format(result_median))

Vector Addition is 1258.9540983606557 times faster than Homomorphic Addition
Median Vector Addition is 1272.4147909967846 times better than Median Homomorphic Addition


In [10]:
# Multiplication is not supported by pallier two ciphertexts are not supported,
# here's an error to add evidence:
result = cipher_num1 * cipher_num2
result = priv_key.decrypt(result)

NotImplementedError: Good luck with that...

In this section, we went through a simple partial homomorphic encryption scheme. We understood the mathematics behind the encryption and decryption and public and private key generation. In the next work, we will be encountering more complex encryption system and their applications in modern world. 